<picture>
<a href="https://sambanova.ai/"\>
<source media="(prefers-color-scheme: dark)" srcset="../../images/SambaNova-light-logo-1.png" height="60">
<img alt="SambaNova logo" src="../../images/SambaNova-dark-logo-1.png" height="60">
</picture>
</a>

# Structured outputs with Instructor and SambaNova

Now you can use [SambaNova Cloud](https://cloud.sambanova.ai/) to generate structured outputs with Instructor.

First, let us import the necessary modules. Please, create a `.env` file at the repository root and include `SAMBANOVA_URL` and `SAMBANOVA_API_KEY` after creating your [SambaNova Cloud API key](https://cloud.sambanova.ai/apis).

Then, create a venv and install the dependencies in the `requirements.txt` in the current folder.

In [ ]:
import os
from pydantic import BaseModel, Field
from typing import List
from openai import OpenAI
import instructor
from dotenv import load_dotenv

load_dotenv('../../.env',override=True)

base_url = os.getenv('SAMBANOVA_URL')
api_key = os.getenv('SAMBANOVA_API_KEY')

In [ ]:

class Character(BaseModel):
    name: str
    fact: List[str] = Field(..., description="A list of multiple facts about the subject")

client = instructor.from_openai(OpenAI(base_url=base_url, api_key=api_key))

resp = client.chat.completions.create(
    model="Meta-Llama-3.1-70B-Instruct",
    messages=[
        {
            "role": "user",
            "content": "Tell me about the company SambaNova Systems.",
        }
    ],
    response_model=Character,
)
print(resp.model_dump_json(indent=2))

# {
#   "name": "SambaNova Systems",
#   "fact": [
#     "SambaNova Systems is a technology company",
#     "It focuses on artificial intelligence and machine learning"
#   ]
# }

# Email classification

In [ ]:
from typing import Literal

client = instructor.from_openai(OpenAI(base_url=base_url, api_key=api_key))

class ClassificationResponse(BaseModel):
    """
    A few-shot example of text classification:

    Examples:
    - "Buy cheap watches now!": SPAM
    - "Meeting at 3 PM in the conference room": NOT_SPAM
    - "You've won a free iPhone! Click here": SPAM
    - "Can you pick up some milk on your way home?": NOT_SPAM
    - "Increase your followers by 10000 overnight!": SPAM
    """

    label: Literal["SPAM", "NOT_SPAM"] = Field(
        ...,
        description="The predicted class label.",
    )


def classify(data: str) -> ClassificationResponse:
    """Perform single-label classification on the input text."""
    return client.chat.completions.create(
        model="Meta-Llama-3.1-70B-Instruct",
        response_model=ClassificationResponse,
        messages=[
            {
                "role": "user",
                "content": f"Classify the following text: <text>{data}</text>",
            },
        ],
    )


if __name__ == "__main__":
    for text, label in [
        ("Hey Jason! You're awesome", "NOT_SPAM"),
        ("I am a nigerian prince and I need your help.", "SPAM"),
        ("Buy cheap watches now!", "SPAM"),
        ("Meeting at 3 PM in the conference room", "NOT_SPAM"),
        ("You've won a free iPhone! Click here", "SPAM"),
        ("Can you pick up some milk on your way home?", "NOT_SPAM"),
        ("Increase your followers by 10000 overnight!", "SPAM")
    ]:
        prediction = classify(text)
        assert prediction.label == label
        print(f"Text: {text}, Predicted Label: {prediction.label}")
        
# Text: Hey Jason! You're awesome, Predicted Label: NOT_SPAM
# Text: I am a nigerian prince and I need your help., Predicted Label: SPAM
# Text: Buy cheap watches now!, Predicted Label: SPAM
# Text: Meeting at 3 PM in the conference room, Predicted Label: NOT_SPAM
# Text: You've won a free iPhone! Click here, Predicted Label: SPAM
# Text: Can you pick up some milk on your way home?, Predicted Label: NOT_SPAM
# Text: Increase your followers by 10000 overnight!, Predicted Label: SPAM